<a href="https://colab.research.google.com/github/wyk172899/StudyMachineLearning/blob/main/%08StudyMachine7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import gdown
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [52]:
def get_page_cnt(isbn):
  url = 'http://www.yes24.com/Product/Search?domain=ALL&query={}'
  r = requests.get(url.format(isbn))
  soup = BeautifulSoup(r.text,'html.parser')
  prd_link = soup.find('a',attrs={'class':'gd_name'})
  url = 'http://www.yes24.com'+prd_link['href']
  r = requests.get(url)
  soup = BeautifulSoup(r.text,'html.parser')
  prd_detail = soup.find('div', attrs={'id':'infoset_specific'})
  prd_tr_list = prd_detail.find_all('tr')
  for tr in prd_tr_list:
    if tr.find('th').get_text() == '쪽수, 무게, 크기':
      page_td = tr.find('td').get_text()
      break
  return page_td.split()[0]

In [53]:
def get_page_cnt2(row):
  isbn = row['isbn13']
  return get_page_cnt(isbn)

In [54]:
url = 'http://data4library.kr/api/loanItemSrch?format=json&startDt=2021-04-01&endDt=2021-04-30&age=20&authKey=4ba168c54fe61dee178046bbda237a2103470447a8695a4b5447c45a5b12207e'
r = requests.get(url)
data = r.json()
books = []
for d in data['response']['docs']:
  books.append(d['doc'])
books_df = pd.DataFrame(books)

In [55]:
top7_books = books_df.head(7)

In [56]:
page_count = top7_books.apply(lambda row:get_page_cnt(row['isbn13']),axis=1)
print(page_count)

0    330쪽
1    300쪽
2    228쪽
3    340쪽
4    264쪽
5    272쪽
6    396쪽
dtype: object


In [58]:
page_count.name = 'page_count'

In [59]:
top7_with_page_count = pd.merge(top7_books, page_count, left_index=True, right_index=True)
top7_with_page_count

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,class_nm,loan_count,bookImageURL,bookDtlUrl,page_count
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,03810,,813.7,문학 > 한국문학 > 소설,479,https://image.aladin.co.kr/product/19359/16/co...,https://data4library.kr/bookV?seq=5430429,330쪽
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,03810,,813.7,문학 > 한국문학 > 소설,407,https://image.aladin.co.kr/product/24512/70/co...,https://data4library.kr/bookV?seq=5707051,300쪽
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,03810,,813.7,문학 > 한국문학 > 소설,391,https://image.aladin.co.kr/product/19804/82/co...,https://data4library.kr/bookV?seq=5446546,228쪽
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,03810,,813.7,문학 > 한국문학 > 소설,379,https://image.aladin.co.kr/product/24131/37/co...,https://data4library.kr/bookV?seq=5674475,340쪽
4,5,5,아몬드손원평 장편소설,손원평,창비,2017,9788936434267,03810,,813.7,문학 > 한국문학 > 소설,375,http://image.aladin.co.kr/product/16839/4/cove...,https://data4library.kr/bookV?seq=3423096,264쪽
5,6,6,목소리를 드릴게요 :정세랑 소설집,지은이: 정세랑,아작,2020,9791165300005,03810,,813.7,문학 > 한국문학 > 소설,327,https://image.aladin.co.kr/product/22246/11/co...,https://data4library.kr/bookV?seq=5549987,272쪽
6,7,7,피프티 피플 :정세랑 장편소설,지은이: 정세랑,창비,2016,9788936434243,03810,,813.7,문학 > 한국문학 > 소설,326,https://bookthumb-phinf.pstatic.net/cover/113/...,https://data4library.kr/bookV?seq=3391014,396쪽
